<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'> Modificado en 2019-1 al 2023-1 por Equipo Docente IIC2233. </font>
</p>

# Tabla de contenidos

1. [*Networking* con Python](#Networking-con-Python)
    1. [Arquitectura cliente-servidor](#Arquitectura-cliente-servidor)
    2. [`Sockets`](#Sockets)
        1. [Cliente TCP](#Cliente-TCP)
        2. [Servidor TCP](#Servidor-TCP)

# *Networking* con Python

**Recuerda que al leer este documento directamente desde GitHub, no estás aprovechando su contenido al 100%.
Editando el código puedes aprender más que leyéndolo. Te recomendamos descargarlo y ejecutarlo desde tu computador para que puedas hacer modificaciones e interactuar con el código.**

**IMPORTANTE: El siguiente material incluye ejemplos que deberían ser ejecutados en programas separados, donde uno actúa como el cliente, y otro actúa como el servidor. Para ello, copia los códigos correspondientes en archivos `.py` en tu computador y ejecútalo en consolas separadas. Si ejecutas los códigos directamente en este `jupyter-notebook` los programas se pueden quedar pegados esperando una conexión.**

En este material, veremos cómo implementar programas en Python para que puedan comunicarse con otro programa, el cual puede encontrarse en el mismo computador o en otro.

## Arquitectura cliente-servidor

Una **arquitectura** en computación es una manera de conectar o construir _hardware_ o _software_ para que cumpla ciertos estándares. La **arquitectura cliente-servidor** es una manera de conectar dos o más programas donde uno cumple el **rol de servidor**, el cual **ofrece servicios** a otros programas que poseen el **rol de cliente**. Los programas cliente **efectúan solicitudes al servidor** y **consumen sus servicios**.

En esta arquitectura el **cliente** es quien **inicia la conexión**, y establece una conexión con el **servidor** de acuerdo a algún protocolo. Por otro lado, el **servidor**, al iniciarse, se dedica a esperar y **escuchar conexiones entrantes** de clientes, y reacciona a cada una de ellas. Al recibir una solicitud de conexión, el servidor puede decidir si la acepta o no, efectúa una acción predefinida, y envía una respuesta al cliente según lo diga el protocolo de comunicación. La transmisión puede hacerse vía TCP ó UDP.

![Fuente: http://en.wikipedia.org/wiki/Client%E2%80%93server_model](imgs/client-server-model.png)
Fuente de la imagen: [Wikipedia](http://en.wikipedia.org/wiki/Client%E2%80%93server_model)

## *Sockets*

Un ***socket*** es un objeto del sistema operativo que permite a un programa transmitir y recibir datos desde y hacia otro programa corriendo en otra máquina, o en la misma máquina pero en otro puerto. Son nuestro "punto de entrada y salida a/desde la red" en nuestro código. Para obtener un *socket*, necesitamos (1) especificar qué tipo dirección IP usaremos (IPv4 o IPv6), y qué protocolo de transporte usaremos (TCP ó UDP).

En Python, los *sockets* son provistos por el módulo `socket`. Para crear un *socket*, debemos crear una instancia de la clase `socket(family, type)`, donde `family` es el tipo de dirección IP que usaremos, y `type` es el protocolo de transporte. Algunos de los valores permitidos son:

- `family`
    - `AF_INET` para direcciones IPv4
    - `AF_INET6` para direcciones IPv6
- `type`
    - `SOCK_STREAM` para TCP
    - `SOCK_DGRAM` para UDP

Por ejemplo, para crear un *socket* TCP con IPv4, lo hacemos de la siguiente manera.

In [1]:
import socket

sock = socket.socket(family=socket.AF_INET, type=socket.SOCK_STREAM)

### Ejemplo: Sockets TCP

#### Cliente TCP

Para establecer una conexión TCP necesitamos (1) la dirección IP del *host* destinatario, y (2) el puerto del *host* destinatario. Para conectarse a un servidor TCP usamos el método `connect((host, port))` del *socket*. El método recibe una tupla, cuyo primer elemento debe ser una dirección IP acorde a lo que especificamos al momento de crear el *socket*, y el segundo elemento es el puerto al que nos queremos conectar. 

Opcionalmente, el primer elemento de la tupla puede ser un *string* con el *hostname*, como por ejemplo, `python.org`, `github.com` o `canvas.uc.cl`. En ese caso, el sistema utilizará el protocolo DNS para obtener la  dirección IP correspondiente a ese *hostname*.

Por ejemplo, hagamos un cliente que se conecte a `146.155.123.21` en el puerto 80 vía TCP.

In [2]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect(('146.155.123.21', 80))

Si nos queremos conectar a [canvas.uc.cl](http://canvas.uc.cl) también lo podemos hacer sin conocer su dirección IP.

In [3]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect(('canvas.uc.cl', 80))

Llegó la hora de enviar algún mensaje. Los *sockets* sólo transmiten *bytes*, por lo que cualquier cosa que enviemos debemos codificarla. Para enviar los *bytes* de un mensaje, se puede ocupar uno de los siguientes métodos de socket:

- `send(bytes)`: Trata de enviar el mensaje almacenado en la variable *bytes*. Sin embargo, puede que envie solo parte de este en vez del mensaje completo, o el envío resulte en error. Debido a lo anterior, este método retorna la cantidad de *bytes* que fue efectivamente enviada, dejando como responsabilidad al programador el llamar nuevamente `send` para enviar los *bytes* restantes hasta que se haya enviado el mensaje completo. Nos daremos cuenta de esto cuando la suma de la cantidad de bytes enviados en cada llamada al método sea igual al largo del mensaje original.
- `sendall(bytes)`: Este método se asegura de enviar todos los *bytes* del mensaje, haciendo uso por debajo del método `send` hasta que el mensaje se haya enviado en su totalidad. Sin embargo, en caso de haber un error en el envío, este método solo lanzará un error, por lo que no podremos saber cuántos bytes efectivamente se habían enviado de forma previa a que ocurriera error.

En el siguiente ejemplo, los *bytes* contienen un mensaje en el formato HTTP, y corresponde a una **solicitud HTTP** (*HTTP Request*) que puede ser comprendida por un servidor web. Los enviaremos mediante el método `sendall`:

In [4]:
solicitud = 'GET / HTTP/1.1\n\n\n'
sock.sendall(solicitud.encode('ascii'))

¿Qué nos habrá respondido el servidor? Para saberlo, debemos ocupar el método `recv(buffer)`, que nos retornará un objeto `bytes`. El parámetro que recibe este método es la cantidad __máxima__* de *bytes* que será leída. La documentación sugiere colocar una potencia de 2 pequeña, como 4096.

*Es la cantidad máxima, porque lo que hace es sacar los bytes que hayan llegado al socket en ese instante. Si es que en el socket habían llegado 20 bytes al momento de llamar a `sock.recv(4096)`, entonces el método me entregará solo 20 bytes.

In [6]:
data = sock.recv(4096)
print(data.decode('ascii'))

El último paso, pero no el menos importante: **debemos cerrar la conexión**; de lo contrario, si nuestro programa continúa, estará ocupando recursos que no utiliza.

In [7]:
sock.close()

Hagamos el ejercicio completo en una sola celda. Además, aprovechemos de manejar las excepciones pues, en *networking*, es muy probable que se produzcan errores. La clase `ConnectionError` es la clase base para distintos errores relacionados con problemas de conexión, por lo que capturando su excepción podremos responder a cualquier problema que ocurra en este sentido.

In [8]:
# Esta celda puede ser ejecutada desde el notebook sin problemas

import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

host = "iic2333.ing.puc.cl"
port = 80

try:
    sock.connect((host, port))
    sock.sendall('GET / HTTP/1.1\nHost: iic2333.ing.puc.cl\n\n'.encode('utf-8'))
    data = sock.recv(4096)
    print(data.decode('utf-8'))
except ConnectionError as e:
    # ConnectionError es la clase base de BrokenPipeError, ConnectionAbortedError,
    # ConnectionRefusedError y ConnectionResetError
    print("Ocurrió un error.")
finally:
    # ¡No olvidemos cerrar la conexión!
    sock.close()

HTTP/1.1 200 OK
Server: nginx/1.14.0 (Ubuntu)
Date: Sat, 07 Oct 2023 00:22:57 GMT
Content-Type: text/html
Content-Length: 11845
Last-Modified: Wed, 17 Aug 2022 05:04:47 GMT
Connection: keep-alive
ETag: "62fc76ef-2e45"
Accept-Ranges: bytes

<!DOCTYPE html><html lang="es"><head><!--[if IE]><!--Get a real browser!--><![endif]--><!--Do NOT read this file, the source is available--><meta charset="utf-8" /><title>IIC2333 Sistemas Operativos y Redes</title><meta content="IIC2333 Sistemas Operativos y Redes, 2022-2" name="description" /><meta content="Cristian Ruz" name="author" /><meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport" /><link href="//cdnjs.cloudflare.com/ajax/libs/twitter-bootstrap/4.0.0-alpha/css/bootstrap.min.css" rel="stylesheet" /><link rel="stylesheet" href="//cdnjs.cloudflare.com/ajax/libs/highlight.js/9.12.0/styles/zenburn.min.css"><script src="../js/highlight.pack.js"></script><script src="../js/fairyDustCursor.js"></script><link href="ht

La respuesta que obtenemos del servidor web ubicado en el puerto 80 del `iic2333.ing.puc.cl` es una **respuesta HTTP** (*HTTP Response*) que contiene 4096 *bytes*. La respuesta HTTP incluye el código HTML que describe el contenido de la página http://iic2333.ing.puc.cl (un **muy buen curso**).

El puerto 80 es usado por los servidores. Se trata un servicio de **texto plano**, esto es, sin encriptación. La mayoría de los servidores web modernos utilizan una versión de HTTP con encriptación, conocida como HTTPS y que funciona en el puerto 443. Conectarse a un servidor web con HTTPS requiere operaciones de encriptación sobre el *socket* que complicarían innecesariamente este ejemplo.

#### Servidor TCP

Un servidor debe escuchar las conexiones que le llegan a él en un puerto específico. Para ello, debemos "enlazar" o "asociar" un *socket* a ese puerto con el método `bind((host, port))`. El primer elemento de la tupla es el *hostname* del *host* en que estamos corriendo el servidor; el segundo elemento es el puerto donde queremos escuchar las conexiones.

Supongamos que queremos escuchar en el puerto 9000.

In [10]:
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Así podemos obtener el hostname de la máquina actual
host = socket.gethostname()
port = 9000

sock.bind((host, port))

En el cliente no es necesario hacer un *bind* entre el *host* y el puerto, ya que el sistema operativo lo hace implícitamente a través del método `connect`, asignando al cliente un puerto aleatorio. Si el servidor al cual nos conectamos exigiera que el puerto de cada cliente esté en un rango específico, tendríamos que "enlazar" el cliente a un puerto específico también.

Ahora, empecemos a escuchar conexiones con el método `listen()`.

In [12]:
sock.listen()

Ahora, empecemos a aceptar conexiones. Para ello, ocupamos el método `accept()`, que nos retornará un _socket_ con el que podremos comunicarnos con ese cliente en específico, y la dirección de ese cliente (en específico, una tupla con su dirección IP y su puerto). El método esperará hasta que haya una conexión entrante; por lo tanto, si nadie se conecta, el programa quedará congelado.

In [13]:
while True:
    socket_cliente, address = sock.accept()
    print("Conexión aceptada desde", address)
    socket_cliente.sendall("Gracias por conectarte\n".encode("utf-8"))
    socket_cliente.close()

Cuando queramos terminar el programa servidor, debemos cerrar el *socket* que estaba aceptando las conexiones.

In [ ]:
sock.close()

Hagamos el ejercicio completo en una sola celda. Supondremos que queremos que el servidor se cierre después de aceptar cinco clientes.

In [ ]:
## No ejecutar esto desde esta celda
## Cree un archivo servidor.py con este código

import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Así podemos obtener el hostname de la máquina actual
host = socket.gethostname()
port = 9000

sock.bind((host, port))
sock.listen()

counter = 0
while counter < 5:
    try:
        socket_cliente, address = sock.accept()
        print("Conexión aceptada desde", address)
        socket_cliente.sendall("Gracias por conectarte\n".encode("utf-8"))
        socket_cliente.close()
        counter += 1
    except ConnectionError:
        print("[Servidor] Ocurrió un error.")

sock.close()

El siguiente código corresponde a un posible cliente para el servidor implementado en la celda anterior.

In [ ]:
## No ejecutar esto desde esta celda
## Cree un archivo cliente.py con este código, y ejecútelo en una 
## ventana/terminal distinta a aquella en que está el servidor

import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Obtenemos el nombre de la máquina a la que nos queremos conectar
# En este caso, nos queremos conectar a esta misma máquina
host = socket.gethostname()
port = 9000

try:
    sock.connect((host, port))
    data = sock.recv(4096)
    print(data.decode("utf-8"))
except ConnectionError as e:
    print("[Cliente] Ocurrió un error")
finally:
    sock.close() 

**NOTA: Junto a este archivo se encuentra la carpeta `ejemplo_base`, donde se encuentran los archivos para el cliente y el servidor utilizando TCP. Recuerda que para que funcione, debes primero ejecutar el archivo del servidor, y luego el del cliente.**